## Minimal example for swaption delta

I try to calculate the DV01 of a swap and swaption to the interest rate. I have observed that for unknown reasons the Swap does have a DV01 whereas the Swaption calculates to a DV01 of 0. This doesn't make any sense in my opinion.

#### Set up market data 
I'm working in a single curve environment with a flat interest rate curve. I also work with a single volatility value that represents the entire swaption vola surface.

In [1]:
from typing import List
import QuantLib as ql

today = ql.Date(10, ql.May, 2019)
ql.Settings.instance().evaluationDate = today

In [2]:
day_count = ql.Actual360()
calendar = ql.TARGET()
business_day_convention = ql.ModifiedFollowing
date_generation = ql.DateGeneration.Forward
end_of_month = False

In [3]:
flat_ois_quote = ql.SimpleQuote(0.04)
flat_ois_quote_handle = ql.QuoteHandle(flat_ois_quote)
ois_curve_handle = ql.YieldTermStructureHandle(ql.FlatForward(today, flat_ois_quote_handle, day_count))

index = ql.Euribor3M(ois_curve_handle)

In [4]:
ir_vol_quote = ql.SimpleQuote(0.2)
ir_vol_handle = ql.QuoteHandle(ir_vol_quote)

#### Creating a forward starting Swap

I'm creating a forward starting swap that can be evaluated on its own and serves as underlying for the swaption. The swap is at par.
Right now the only way I'm aware of to create a par swap is to create an equivalent dummy swap with a dummy fixed rate, extract its fair rate and then create a new par swap with the fair rate as its fixed rate.

In [4]:
notional = 1000000
tts = 1  # time to start in years
ttm = 2  # time to maturity in years
dummy_fixed_rate = 0.02

start_date = calendar.advance(today, tts, ql.Years)
maturity_date = calendar.advance(today, ttm, ql.Years)

In [6]:
# Creating the float leg schedule
float_leg_tenor = ql.Period(3, ql.Months)
float_schedule = ql.Schedule(start_date, maturity_date, float_leg_tenor, calendar, business_day_convention, business_day_convention, date_generation, end_of_month)

In [7]:
# Creating the fixed leg schedule
fixed_leg_tenor = ql.Period(3, ql.Months)
fixed_schedule = ql.Schedule(start_date, maturity_date, fixed_leg_tenor, calendar, business_day_convention, business_day_convention, date_generation, end_of_month)

In [8]:
# Creating the super simple single curve pricing engine
pricing_engine = ql.DiscountingSwapEngine(ois_curve_handle)

In [9]:
#creating the dummy swap and calculating its fair rate
direction = ql.VanillaSwap.Receiver
dummy_swap = ql.VanillaSwap(direction, notional, fixed_schedule, dummy_fixed_rate, day_count, float_schedule, index, spread=0, floatingDayCount=day_count)
dummy_swap.setPricingEngine(pricing_engine)
fair_rate = dummy_swap.fairRate()
print(fair_rate)
print(dummy_swap.NPV())

0.040203012205051666
-19122.912717654242


In [10]:
#creating the real swap using the calculated fair rate
swap = ql.VanillaSwap(direction, notional, fixed_schedule, fair_rate, day_count, float_schedule, index, spread=0, floatingDayCount=day_count)
swap.setPricingEngine(pricing_engine)
print(swap.NPV())

7.275957614183426e-12


#### Creating the Swaption
The swaptions exercise date is on the start date of the forward starting swap that is the swaptions underlying.

In [11]:
swaption_pricing_engine = ql.BlackSwaptionEngine(ois_curve_handle, ir_vol_handle)
exerciseDate = today+ ql.Period(1, ql.Years)
exercise = ql.EuropeanExercise(exerciseDate)
swaption = ql.Swaption(swap, exercise)
swaption.setPricingEngine(swaption_pricing_engine)
print(swaption.NPV())

3035.3261267062176


#### Finite difference calculator
To facilitate sensitivity calculation I set up a quick utility function that calculates a sensitivity with finite differences.

In [12]:
def fd_sensitivity(quotes: List[ql.SimpleQuote], instrument: ql.Instrument):
    abs_bump = 0.0001
    p0 = instrument.NPV()
    origVals = []
    for quote in quotes:
        origVals.append(quote.value())
        quote.setValue(quote.value()+abs_bump)
    p1 = instrument.NPV()
    finite_difference = (p1-p0)/abs_bump
    origVals.reverse()
    for quote in quotes:
        quote.setValue(origVals.pop())
    return finite_difference

#### Calculating Swap Delta

In [14]:
print(fd_sensitivity([flat_ois_quote], swap))

-956017.1435484517


#### Calculating Swaption Delta

In [15]:
print(fd_sensitivity([flat_ois_quote], swaption))


-442505.598476323
